In [2]:
import torch
import numpy as np
from PIL import Image
import pdb
import os
import cv2
from scipy.io import loadmat,savemat
from collections import defaultdict
import tqdm

/home/zhengwei/anaconda3/envs/yzwT2V/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_path = "/home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K"
train_style = ['A', 'B', 'C', 'D', 'E', 'F']
files_rgb = os.listdir(data_path+'/photo/train')
files_sk = {s: os.listdir(f'{data_path}/sketch/{s}/train') for s in train_style}

In [4]:
pid_container = set()
for s in files_sk.keys():
    files = files_sk[s]
    for img_path in files:
        pid = int(img_path[:4])
        pid_container.add(pid)
pid2label = {pid:label for label, pid in enumerate(pid_container)}


_train_image = defaultdict(list)
_train_sketch = defaultdict(list)

sketch2idx = {s: i for i, s in enumerate(train_style)}

for s in files_sk.keys():
    train_image = files_sk[s]
    for img_path in train_image:
        pid = int(img_path[:4])
        _train_image[pid].append(f'sketch/{s}/train/{img_path}')
        _train_sketch[pid].append(sketch2idx[s])


In [58]:
print(_train_sketch)

defaultdict(<class 'list'>, {736: [0, 2, 4, 5], 487: [0, 1, 2, 4, 5], 804: [0, 2, 4, 5], 448: [0, 1, 2, 4, 5], 607: [0, 2, 3, 4, 5], 911: [0, 2, 4, 5], 932: [0, 2, 4, 5], 163: [0, 2, 3, 4, 5], 183: [0, 2, 3, 4, 5], 824: [0, 2, 4, 5], 731: [0, 2, 4, 5], 260: [0, 1, 2, 3, 4, 5], 750: [0, 2, 4, 5], 310: [0, 1, 2, 3, 4, 5], 983: [0, 2, 4, 5], 351: [0, 1, 2, 3, 4, 5], 25: [0, 2, 3, 4, 5], 923: [0, 2, 4, 5], 320: [0, 1, 2, 3, 4, 5], 1047: [0, 2, 4, 5], 866: [0, 2, 4], 1037: [0, 2, 4, 5], 1008: [0, 2, 4, 5], 247: [0, 1, 2, 3, 4, 5], 716: [0, 2, 4, 5], 165: [0, 2, 3, 4, 5], 740: [0, 2, 4, 5], 302: [0, 1, 2, 3, 4, 5], 103: [0, 2, 3, 4, 5], 438: [0, 1, 2, 4, 5], 40: [0, 2, 3, 4, 5], 305: [0, 1, 2, 3, 4, 5], 381: [0, 1, 2, 3, 4, 5], 44: [0, 2, 3, 4, 5], 120: [0, 2, 3, 4, 5], 906: [0, 2, 4, 5], 693: [0, 2, 4, 5], 289: [0, 1, 2, 3, 4, 5], 728: [0, 2, 4, 5], 391: [0, 2, 3, 4, 5], 373: [0, 1, 2, 3, 4, 5], 846: [0, 2, 4, 5], 977: [0, 2, 4, 5], 834: [0, 2, 4, 5], 288: [0, 1, 2, 3, 4, 5], 805: [0, 2, 4,

In [5]:
import shutil

def copy_file(src_path, dst_path):
    shutil.copy(src_path, dst_path)

In [6]:
# seed manual seed for random function
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

few_shot_basepath = '/home/zhengwei/Desktop/Zhengwei/Projects/datasets/Market-Sketch-1K/sketch/fewshot'

for pid in tqdm.tqdm(sorted(_train_image.keys())):
    if not pid in pid2label.keys():
            continue
    img_paths = _train_image[pid]
    sketch_idx = _train_sketch[pid]

    # print(img_paths)

# select a random number from 0 to len(sketch_idx)
    select = np.random.randint(0, len(sketch_idx))
    selected_sketch = sketch_idx[select]
    unselected_sketch = [sketch_idx[i] for i in range(len(sketch_idx)) if i != select]

    # print(sketch_idx)
    # print(selected_sketch)
    # print(unselected_sketch)
    
    for idx, img_path in enumerate(img_paths):
        style = train_style[sketch_idx[idx]]
        img = img_path.split('/')[-1][:4]
        if idx == select:
            # copy image_file from one path to another, with out open it
            copy_file(f'{data_path}/{img_path}', f'{few_shot_basepath}/train/finetune/{img}_{style}.jpg')
        else:
            copy_file(f'{data_path}/{img_path}', f'{few_shot_basepath}/train/test/{img}_{style}.jpg')
    # break




100%|██████████| 498/498 [00:00<00:00, 1438.17it/s]


In [8]:
import os

def print_directory_structure(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for idx, f in enumerate(files):
            if idx < 5:
                print('{}{}'.format(subindent, f))

# Replace 'your_directory_path' with the path of the directory you want to print
print_directory_structure(few_shot_basepath)

fewshot/
    train/
        finetune/
            0333_E.jpg
            0303_E.jpg
            0674_C.jpg
            0788_C.jpg
            0557_A.jpg
        test/
            0237_F.jpg
            0833_A.jpg
            0221_C.jpg
            0127_D.jpg
            0390_F.jpg
    all/
        finetune/
            0319_B.jpg
            0846_A.jpg
            0778_C.jpg
            0200_E.jpg
            0333_E.jpg
        test/
            0532_D.jpg
            0237_F.jpg
            0668_E.jpg
            0080_C.jpg
            0462_C.jpg
